# El objetivo de este notebook es leer el/los fichero(s) que estén unbicados en un ruta derteminada en formato .GPX* y prepararlos para su posterior utilización.

## En el proceso se enriquecen con información adicional externa a los ficheros, y se normalizan los datos comunes con el fin de poder hacerlos interaccionar.






##### *GPX, o GPS eXchange Format (Formato de Intercambio GPS) es un esquema XML pensado para transferir datos GPS entre aplicaciones. Se puede usar para describir puntos (waypoints), recorridos (tracks), y rutas (routes). (Wikipedia)




In [2]:
#Parte de Importación de paquetes
from xml.dom.minidom import parse
import xml.dom.minidom

from os import listdir, rename
from os.path import isfile, join
import time

import pandas as pd
import math as mt

from scipy.signal import savgol_filter

from geopy.distance import distance
from geopy import Point

import ipynb.fs.full.Weather_History as weather

# Funciones para ENRIQUECER los FICHEROS.
COMUN PARA TODOS LOS ORIGENES

In [10]:
# (0,360) = Norte
# 90 = Este
# 180 = Sur
# 270 = Oeste

def sync_bearing(ori_bearing):
    """Ajusta en el mismo tipo de escala la dirección de viento.

    Devuelve un número con el valor de los grados de la dirección del viento:
    
    Parámetros:
    ori_bearing -- Dirección de viento original
    """
        
    if ori_bearing < -90:
        new_bearing = 450+ori_bearing
    else:
        new_bearing = ori_bearing+90
    return new_bearing


def func_bearing(bc,bv):
    """Resuelve trigonometricamente, el número por el que hay que multiplicar
       el viento real para obtener la velocidad del viento aparente.
       Si el viento es favorable devuelve un número negativo
       Con vientos en contra devuelve un número positivo

    Devuelve en una tupla el factor multiplicador y la diferencia de angulos:
    
    Parámetros:
    bc -- Ángulo del ciclista
    bv -- Ángulo del viento
        
    """
    if bv+180 > 360:
        bv_mod = bv-180
    else:
        bv_mod = bv+180
    if bc > bv_mod:
        dif = bc-bv_mod
    else:
        dif = bv_mod-bc
        
    multiplicador_viento = mt.cos(dif/(180/mt.pi))
    return(multiplicador_viento, dif)

def deteccion_tramos(input_df):
    """Funcion que detecta el puntos continuos de subida y bajada
    para agruparlos en tramos o rampas.

    Devuelve una tupla con tres tipos de listas de tramos:
        lista_tramos: Lista de tramos continuos por pendiente, con punto de entrada y salida e información sobre el tramo
        tramos: Lista con el número de tramo al que pertenece cada punto del track original
        tramos_100: Lista de tramos segmentado por fracciones de 100m
    
    Parámetros:
    inpuf_df -- Dataframe con los datos enriquecidos procedentes de los fichero gpx
    """
    
    #Almacenamos la prueba de entrenamiento que vamos a procesar
    prueba = input_df.iloc[1]["prueba"]

    #Inicializacon las varibles de control y resultado del proceso
    tramos = [0]
    tramo = 0
    pendiente = 0
    pendiente_ant = 0

    tramos_100 = [0]
    tramo100 = 0
    acum_dist = 0
        
    lista_tramos = []
    tramo_inicial = 0
    power_acum = 0
    velocidad_aparente_acum = 0
    
    #Inicializamos el iterador del WHILE para que empiece en 1, y vaya comparando con el anterior
    i = 1
    
    #Inicializamos el comienzo del dataframe, ya que los primeros registros suelen ser
    #irregulares y así evitar problemas con los cálculos aritméticos
    
    input_df.iloc[0,5] = 0 #Dist=0 para el primer punto
    input_df.iloc[1,5] = input_df.iloc[1,11] #Dist_calculada = Dist
    
    tiempo_parado = 0
    tiempo_parado_punto = 0

    #Recorremos cada track point del dataframe
    print("Entramos en el While1 %d" % len(input_df.index))
    while i < len(input_df.index):
  
                
        #Calculo de tramos continuos
        
        #Calculamos la variación de altitud entre el punto i e i-1, para poder comprobar la tendencia.
        pendiente = input_df.iloc[i]["new_ele"]-input_df.iloc[i-1]["new_ele"]

        
               
        
        #Si no existe un cambio de tendencia respecto al punto anterior y no hemos llegado al final de Dataframe,
        #añadimos el tramo en el que estamos a la variable "tramos".
        if ((pendiente * pendiente_ant >=0) and ((i+1) != len(input_df.index))):
                
            tramos.append(tramo)
        else:
            #En caso contrario, bien por fin de Dataframe como por cambio de tendencia, calculamos los datos necesarios y
            #almacenamos la información del tramo.
            
            #Cada tramo en 1 línea
            #Son intervalos cerrados por la izquierda y abiertos por la derecha
                        
            
            if ((i+1) != len(input_df.index)):
                distancia_tramo = input_df.iloc[i-1]["dist_c"]-input_df.iloc[tramo_inicial]["dist_c"]
                desnivel_tramo = input_df.iloc[i-1]["new_ele"]-input_df.iloc[tramo_inicial]["new_ele"]
            else:
                distancia_tramo = input_df.iloc[i]["dist_c"]-input_df.iloc[tramo_inicial]["dist_c"]
                desnivel_tramo = input_df.iloc[i]["new_ele"]-input_df.iloc[tramo_inicial]["new_ele"]
            
            porcent_des = desnivel_tramo/distancia_tramo*100
            
            sp_vel_aparente = velocidad_aparente_acum/distancia_tramo
                       
            
            lista_tramos.append(\
                                (tramo, tramo_inicial,\
                                 i-1, distancia_tramo,\
                                 sp_vel_aparente,porcent_des,\
                                 input_df.iloc[0]["dist_c"]+acum_dist,prueba))
            
            velocidad_aparente_acum = 0
            
            tramo_inicial = i-1
            tramo +=1
            tramos.append(tramo)
        
        #En cada punto, acumulamos dentro del tramos el producto de la potencia por el tiempo empleado (para obtener al final potencia media)
        #y acumulamos el producto del viento_aparente con la distancia para obtener al final del tramo su valor medio.
        velocidad_aparente_acum = velocidad_aparente_acum +\
                                    (input_df.iloc[i]["viento_aparente"]*(input_df.iloc[i]["dist_c"]-input_df.iloc[i-1]["dist_c"]))
        
        #Actualizamos los valores de las variables para punto
        pendiente_ant = pendiente            

        #Se realiza un cálculo simple para almacenar tramos de 100m
        acum_dist = input_df.iloc[i-1]["dist_c"]
        tramo100 = mt.floor(acum_dist/100)
        tramos_100.append(tramo100)
        
        i +=1
          

    print("Salimos del While")
    print("tramos : %d continuos y %d de 100m" % (len(set(tramos)), len(set(tramos_100))))
   
    return lista_tramos, tramos, tramos_100


def enriquecimiento(list_track):
    """Funcion que enriquece la información de entrada para su posterior análisis
    
    Devuelve una tupla con dos dataframes, uno con la información enriquecida por puntos y otro por tramos:
        df2: Dataframe enriquecido para cada uno de los puntos de gpx
        df_tramos: Dataframe enriqeucido para cada uno de los tramos calculados
    
    Parámetros:
    list_track -- Lista de tuplas con la información de cada punto del fichero gpx
    """
    #Cargamos la lista en un Dataframe
    df = pd.DataFrame(list_track)
    #Definimos los nombres de las columnas
    df.columns = ["timeoffset", "lat", "lon", "alt", "prueba"]
    
    #Rellenamos potencias nulas a 0

    df2= df.copy()
    i=1
    
    prueba = df2.iloc[1]["prueba"]
    
    #Llamamos a la funcion get_viento del Notebook Weather_History que recupera el viento y su dirección para la fecha y la ubicacion indicada
    #viento, bv = weather.get_viento(ubicacion,fecha)
    
    #A efectos de poder simular una prueba futura, para introducir manualmente los valores, se debe comentar la línea anterior
    #y desconmentar las siguientes
    
    viento = 10
    bv = 30
    
    #Inicializamos la variables locales
    
    viento_aparente=[]
    viento_aparente.append(0)

    wind=[]
    wind.append(0)

    mult=[]
    mult.append(0)

    dist_c=[]
    dist_c.append(0)

    ele_gain = []
    ele_gain.append(0)

    d=0
    e=0

    porc=[]
    porc.append(0)

    spd_c=[]
    spd_c.append(0)

    spd_prev=[]
    spd_prev.append(0)

    ele_prev=[]
    ele_prev.append(0)

    bearing=[]
    bearing.append(0)

    timeoffset_prev=[]
    timeoffset_prev.append(0)

    ini_timeoffset = df2.iloc[0]['timeoffset']
    time=[]
    time.append(0)

    #Recorremos los puntos de lista de la prueba que estamos enriqueciendo
    
    while i < len(df2):
        #Fijamos los puntos del registro anterior y del actual.
        punto1 = Point(df2.iloc[i-1]['lon'], df2.iloc[i-1]['lat'])
        punto2 = Point(df2.iloc[i]['lon'], df2.iloc[i]['lat'])
        
        #Almacenamos en una lista el tiempo desde incio
        time.append(df2.iloc[i]['timeoffset']-ini_timeoffset)
        
        #Calculamos mediante la libreria geopy la distancia entre dos puntos en definidos por coordenadas,
        #y las almacenamos en una lista
        d = distance(punto1, punto2).m
        dist_c.append(d)
        
        #Calculamos la velocidad en base a la distancia y el tiempo
        spd_c.append(float(d/(df2.iloc[i]['timeoffset']-df2.iloc[i-1]['timeoffset'])))
        
        #Calculamos la altitud de los puntos i-1 e i, para alamcenar la diferencia de altitud
        punto1 = df2.iloc[i-1]['alt']
        punto2 = df2.iloc[i]['alt']
        e = punto2-punto1
        ele_gain.append(e)
        
        #Calculamos la pendiente
        porc.append(e*100/d)
        
        #Almacenamos en variables locales las coordenadas de los puntos i-1 e i
        lona = df2.iloc[i-1]['lon']
        lata = df2.iloc[i-1]['lat']
        lonb = df2.iloc[i]['lon']
        latb = df2.iloc[i]['lat']
        
        #Se calculan las componentes del vector de desplazamiento
        X = mt.cos(lonb)*mt.sin(latb-lata)
        Y = mt.cos(lona)*mt.sin(lonb)-mt.sin(lona)*mt.cos(lonb)*mt.cos(latb-lata)
        
        #Se regulariza el ángulo de la dirección del viento, y se almacena su factor y velocidad.
        #1 radian = 57.2957795 grados
        bearing.append(sync_bearing(mt.atan2(X,Y)*57.2957795))
        mult.append(func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[0])
        wind.append(func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[1])
        
        #Se calcula el viento_aparente, es la velocidad del viento que le impacta al ciclista de forma frontal, el que tien en contra.
        viento_aparente.append((func_bearing(sync_bearing(mt.atan2(X,Y)*57.2957795),bv)[0])*viento)

        i+=1
        
    #Se construye un Dataframe con las listas de valores calculados previamente
    df2['dist_c']=dist_c
    df2['ele_gain']=ele_gain
    df2['porc']=porc
    df2['spd_c']=spd_c
    df2['bearing']=bearing
    df2['wind']=wind
    df2['mult']=mult
    df2['viento_aparente']=viento_aparente

    
    #Se vuelve a recorrer el Dataframe para calcular diferencias entre velocidades, altitudes y tiempo entre puntos
    i=1
    while i < len(df2):
        spd_prev.append(df2.iloc[i-1]['spd_c'])
        ele_prev.append(df2.iloc[i-1]['alt'])
        timeoffset_prev.append(df2.iloc[i-1]['timeoffset'])
        i+=1
    
    #Se añaden al Dataframe los nuevos campos calculados    
    df2['spd_prev']=spd_prev
    df2['ele_prev']=ele_prev
    df2['timeoffset_prev']=timeoffset_prev

    #Aplicamos el suavizado de señal, con el objetivo de minimizar las imprecisiones generadas por los datos proporcionados
    #por el GPS, y así normalizar en cierta medida aberraciones en los datos, no se considera eliminación de outlayers.
    
    axis_y = df["alt"]
    axis_yhat = savgol_filter(axis_y, 51, 2)
    df2["new_ele"] = axis_yhat
    df2["time_seg"] = time

    #Una vez enriquecido el fichero, realizamos la llamada a la función para detectar tramos
    lista_tramos, tramos, tramos_100 = deteccion_tramos(df2)
    
    #Añadimos al Dataframe dos columnas con el tramo en el que entra cada punto
    df2["tramo"] = tramos
    df2["tramos_100"] = tramos_100

    #Creamos un nuevo Dataframe con la lista de tramos
    df_tramos = pd.DataFrame(lista_tramos)
    df_tramos.columns= ["tramo", "tramo_inicial", "tramo_final","dist","viento_aparente","porc","dist_acum", "prueba"]
    
    #AÑADIMOS NUEVAS VARIABLES DEPENDIENTES DE LA RUTA Y EL CICLISTA
    #En el futuro se usarán para cálculos cuando se entrenen los modelos con datos de distintos ciclistas

    df2['altura'] = path.split('_')[2]
    df2['peso'] = path.split('_')[3]
    df2['tpo_bici'] = path.split('_')[4]
    df2['tpo_entreno'] = path.split('_')[5] 

    return df2,df_tramos


def to_dt(dato):
    """Funcion que tranforma una texto en segundos
    
    Devuelve la hora expresada en segundos
    
    Parámetros:
    dato -- Hora
    """
    t=pd.to_datetime(dato)
    return t.hour*3600+t.minute*60+t.second

## ANALISIS TRACK BASADO EN GPX para varios ficheros
#PASO_1
#Importamos los xml procedente de gpx que esten en la carpeta indicada por path
##########################################

In [11]:
#Ejecución del cuerpo del programa

#Definimos la ruta en la cual están los ficheros
ruta='input_files_sim/'

#Los fichero cuya extensión sea .gpx los almacenamos en una lista iterable
ficheros = [f for f in listdir(ruta) if isfile(join(ruta, f)) & f.endswith(".gpx")]

#Inicializamos la variables dodnde almacenamos los resultados de las pruebas
result_list_tramos = []
result_list = []


#Recorremos los fichero uno a uno
for fichero in ficheros:
    path = join(ruta, fichero)
    
    print('Se inicia el procesado de %s' %fichero)

    #Instanciamos el parser de XML
    DOMTree = xml.dom.minidom.parse(path)
    #Recuperamos los elementos del XML
    collection = DOMTree.documentElement
    
    #Filtros los trackpoint
    trkpts = collection.getElementsByTagName("trkpt")
     
    list_track = []
    count=0

    #Recorremos los trackpoint extrayendo de cada uno de ellos la información que necesitamos
    
    for trkpt in trkpts:
        count +=1
        if trkpt.hasAttribute("lat"):
          lat = float(trkpt.getAttribute("lat"))
        if trkpt.hasAttribute("lon"):
          lon = float(trkpt.getAttribute("lon"))

        if len(trkpt.getElementsByTagName("ele"))==0:
            alt = ''
        else:
            alt = float(trkpt.getElementsByTagName("ele")[0].firstChild.nodeValue)

        if len(trkpt.getElementsByTagName("time"))==0:
            time = ''
        else:
            time = trkpt.getElementsByTagName("time")[0].firstChild.nodeValue

        timeoffset = to_dt(time)
        
        #Descartamos los trackpoint que no tienen la información más relevante para nosotros.
        #if (pwr!='') and (alt!=''):
        if (alt!=''):
            list_track.append([timeoffset, lat, lon, alt, fichero])
    
    #Realizamos la llamada para el enriquecimento del dato.
    df_puntos, df_tramos = enriquecimiento(list_track)
    
    #Añadimos los resultados de la llamada anterior a la variable donde se guardan en una lista todos los Dataframe de las pruebas
    result_list.append(df_puntos)
    result_list_tramos.append(df_tramos)
    
    
    print ("Puntos cargados del fichero %s: %d" %(fichero, count))

#Una vez procesados todas pruebas, concatenamos todos los Dataframe en uno   
result = pd.concat(result_list)
result_tramos = pd.concat(result_list_tramos)

#Exportamos a Excel los resultados
print ("Puntos cargados: %d" %len(list_track))
result.to_excel('Procesado_simulacion.xlsx')
result_tramos.to_excel('Procesado_Tramos_simulacion.xlsx')
print('finish')

Se inicia el procesado de xxxxxxxx_LEVS_195_90_R_E.gpx
Entramos en el While1 2697
Salimos del While
tramos : 85 continuos y 1 de 100m
Puntos cargados del fichero xxxxxxxx_LEVS_195_90_R_E.gpx: 2697
Puntos cargados: 2697
finish


In [5]:
list_track

[[39641,
  '',
  '',
  '',
  '',
  '',
  40.98826,
  -3.637238,
  988.071,
  '',
  'ecotrimad-olimpico.gpx'],
 [39712,
  '',
  '',
  '',
  '',
  '',
  40.988294,
  -3.637358,
  992.041,
  '',
  'ecotrimad-olimpico.gpx'],
 [39719,
  '',
  '',
  '',
  '',
  '',
  40.988189,
  -3.637342,
  994.015,
  '',
  'ecotrimad-olimpico.gpx'],
 [39724,
  '',
  '',
  '',
  '',
  '',
  40.988077,
  -3.637338,
  994.008,
  '',
  'ecotrimad-olimpico.gpx'],
 [39728,
  '',
  '',
  '',
  '',
  '',
  40.987974,
  -3.637312,
  995.018,
  '',
  'ecotrimad-olimpico.gpx'],
 [39731,
  '',
  '',
  '',
  '',
  '',
  40.987877,
  -3.63729,
  995.014,
  '',
  'ecotrimad-olimpico.gpx'],
 [39734,
  '',
  '',
  '',
  '',
  '',
  40.987769,
  -3.637273,
  996.076,
  '',
  'ecotrimad-olimpico.gpx'],
 [39737,
  '',
  '',
  '',
  '',
  '',
  40.98764,
  -3.637235,
  996.029,
  '',
  'ecotrimad-olimpico.gpx'],
 [39739,
  '',
  '',
  '',
  '',
  '',
  40.987542,
  -3.637238,
  996.086,
  '',
  'ecotrimad-olimpico.gpx'],
 [39